Classificação de Texto Multilabel


José Augusto de Almeida Neto


## Importar bibliotecas


In [1]:
import time
import pandas as pd
import torch
from torch.utils.data import DataLoader
from transformers import BertForSequenceClassification, BertTokenizer

c:\VSCODEProjects\artigo2024\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Carregar datasets


In [2]:
# Carregar dataset
name = 'teste2'
df_case_study = pd.read_csv(f'../datasets/dataset-case_study-{name}.csv')
df_case_study

,restaurante_name,user_name,review_date,stars,comment_text,year,platform,language,comment_text_translated
0,cipriani,Viva Gastronomia,9 anos atrás no Zomato,1,Hotelaria e gastronomia são duas artes demand...,2015,Zomato,pt,hotelaria e gastronomia são dua art demandam m...
1,cipriani,Viva Gastronomia,9 anos atrás no Zomato,1,Hotelaria e gastronomia são duas artes demand...,2015,Zomato,pt,o hotel copacabana palac é uma referência para...
2,cipriani,Viva Gastronomia,9 anos atrás no Zomato,1,Hotelaria e gastronomia são duas artes demand...,2015,Zomato,pt,"à part da ambientação , as opçõ gastronômica c..."
3,cipriani,Viva Gastronomia,9 anos atrás no Zomato,1,Hotelaria e gastronomia são duas artes demand...,2015,Zomato,pt,o carpaccio cipriani é um item no qual erro nã...
4,cipriani,Viva Gastronomia,9 anos atrás no Zomato,1,Hotelaria e gastronomia são duas artes demand...,2015,Zomato,pt,o couvert entrega variedad de pãe e molho que ...
...,...,...,...,...,...,...,...,...,...
13728,tangara,Maria Lins,4 anos atrás no Google,1,Comida divina,2020,Google,it,comida divina
13729,tangara,Ricardo Polisel Alves,5 anos atrás no Foursquare,0,Tasting Menu was indeed very good. Service wa...,2019,Foursquare,en,o menu de degustação foi realment muito bom
13730,tangara,Ricardo Polisel Alves,5 anos atrás no Foursquare,0,Tasting Menu was indeed very good. Service wa...,2019,Foursquare,en,"os vinho começam acima de r $ 250 , ma , no en..."
13731,tangara,Ricardo Polisel Alves,5 anos atrás no Foursquare,0,Tasting Menu was indeed very good. Service wa...,2019,Foursquare,en,o ambient era aconchegant e as mesa estavam ad...


In [3]:
labels = ['ambiente', 'bebida', 'comida', 'geral',
          'localização', 'outros', 'preço', 'serviço']

In [4]:
# Renomeando a coluna de df
df = df_case_study[['comment_text_translated']].copy()
df.rename(columns={'comment_text_translated': 'sentenca'}, inplace=True)
df

,sentenca
0,hotelaria e gastronomia são dua art demandam m...
1,o hotel copacabana palac é uma referência para...
2,"à part da ambientação , as opçõ gastronômica c..."
3,o carpaccio cipriani é um item no qual erro nã...
4,o couvert entrega variedad de pãe e molho que ...
...,...
13728,comida divina
13729,o menu de degustação foi realment muito bom
13730,"os vinho começam acima de r $ 250 , ma , no en..."
13731,o ambient era aconchegant e as mesa estavam ad...


## Hiperparâmetros


In [5]:
# Hiperparâmetros
max_len = 128
test_batch_size = 4
num_labels = len(labels)
threshold = 0.28

## Funções auxiliares


In [6]:
class TokenizeDataset(torch.utils.data.Dataset):

    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.df = df
        self.title = df['sentenca']
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())

        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten()
        }

In [7]:
def run_prediction(model, df_test):
    start_time = time.time()

    # Carregar o tokenizador BERT
    tokenizer = BertTokenizer.from_pretrained(
        'neuralmind/bert-base-portuguese-cased')

    # Tokenizar o dataset
    tokenized_test_data = TokenizeDataset(df_test, tokenizer, max_len)

    # Criar DataLoader para carregar os dados em lotes durante o treinamento
    test_loader = DataLoader(tokenized_test_data,
                              batch_size=test_batch_size, shuffle=True)

    model.eval()
    all_predictions = []
    sigmoid = torch.nn.Sigmoid()
    with torch.no_grad():
        for batch in test_loader:
            inputs_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            token_type_ids = batch['token_type_ids']

            # Passagem Direta (inferência)
            outputs = model(input_ids=inputs_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

            # A função sigmóide é aplicada aos logits para converter os valores em probabilidades
            logits = outputs.logits
            predicted_probs = sigmoid(logits)

            # Aplicação do threshold
            predicted_labels = (predicted_probs > threshold).float()

            # Conversão dos rótulos previstos para numpy arrays
            predicted_labels_np = predicted_labels.cpu().numpy()

            # Coleção das predictions
            all_predictions.extend(predicted_labels_np)
    
    # Calcular o tempo total de teste
    end_time = time.time()
    elapsed_time = end_time - start_time
    hours, rem = divmod(elapsed_time, 3600)
    minutes, seconds = divmod(rem, 60)

    print(
        f"Tempo de predição: {int(hours):02}:{int(minutes):02}:{seconds:.2f}")

    return all_predictions

## Aplicação do modelo

In [8]:
# Importar o modelo
model_dir = './bertimbau-model'
model = BertForSequenceClassification.from_pretrained(model_dir)

In [9]:
name = 'teste2'

# Fazer previsões no dataset sem anotações
predictions = run_prediction(model, df)

# Salvar as previsões em um arquivo CSV
predictions_df = pd.DataFrame(predictions, columns=labels)
predictions_df.to_csv(f'../datasets/dataset-case_study_predictions-{name}.csv', index=False)

c:\VSCODEProjects\artigo2024\venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Tempo de predição: 00:54:16.82
